# Import libraries

In [ ]:
# General
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.lines import Line2D 
from matplotlib.font_manager import FontProperties
import seaborn as sns
import geopandas as gpd
%matplotlib inline

# Webscraping
from dateutil import rrule
from datetime import datetime, timedelta, date
from locale import atof, setlocale, LC_NUMERIC

# Sklearn
import sklearn
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans


# Statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#from statsmodels.tsa.ar_model import AutoReg
import statistics as stats

# Data Collection and Preprocessing
***
The main datasources are historical bond holdings data as provided by the ECB for the Corporate Sector Purchase Program (CSPP). Data for Industry Sector Codes as provided by Compustat and Greenhouse Gas Data from Climate Watch and the International Energy Agency. 
***
This notebook contains the process of collecting the data from different sources, cleaning is and preprocessing it for further analysis. 
***

## CSPP Holdings Data
***
In this section of the notebook historical data on the purchases of the ECB within the CSPP are scraped from the ECB website. This consists of two sources in particular:

1. **Historical lists of securities held under the CSPP**
    - these are csv files that are published weekly starting 2017 <br>
    - content: individual bonds that are purchased in a particular week <br>
    - auxiliary data provided: national bank that purchased bonds, issuer, maturity rate and coupon rate <br>
    - not provided: purchase volume of individual bonds or weekly puchase volumes in general <br>
<br>
2. **History of cumulative purchase breakdowns under the CSPP**
    - one breakdown file with monthly holdings in CSPP program (primary & secondary market holdings) 

***

### Scrape historical lists of securities held under CSPP

***
This section includes a class to scrape and combine all csv files that contain historical securities 
***

In [ ]:
# Class for webscraping
class Get_CSPP():
    
    def __init__(self, startDate, endDate, base_url_old, base_url_new, end_url):
        self.startDate = startDate        
        self.endDate = endDate
        self.base_url_old = base_url_old
        self.base_url_new = base_url_new
        self.end_url = end_url

    # function to get datespan (bonds data is released every 7 days)
    def datespan(self):
        delta = timedelta(days = 7)
        currentDate = self.startDate
        while currentDate < self.endDate:
            yield currentDate
            currentDate += delta

    # function to obtain csv data from ECB website
    def cssp_collect(self):
        colnames = ['NCB','ISIN','ISSUER','MATURITY DATE','COUPON RATE']
        dfs_old = [] 
        dfs_new = [] 
        for day in Get_CSPP.datespan(self):
        
            try:
                url_template_old = self.base_url_old + day.strftime("%Y%m%d") + self.end_url
                data = pd.read_csv(url_template_old, sep = ',', encoding = 'latin-1')
                new_cols = {x: y for x, y in zip(data.columns, colnames)}
                data = data.rename(columns = new_cols)
                data.insert(1, "Date", day)
                data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
                dfs_old.append(data)

            except:
          
                try:
                    url_template_new = self.base_url_new + day.strftime("%Y%m%d") + self.end_url
                    data = pd.read_csv(url_template_new, sep = ',', encoding = 'latin-1')
                    new_cols = {x: y for x, y in zip(data.columns, colnames)}
                    data = data.rename(columns = new_cols)
                    data.insert(1, "Date", day)
                    data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
                    dfs_new.append(data)  

                except:
                     pass

        combined_dfs = [*dfs_old, *dfs_new]
        return pd.concat(combined_dfs, ignore_index = True)

In [ ]:
# obtain complete cspp data
base_url_new = 'https://www.ecb.europa.eu/mopo/pdf/CSPP_PEPP_corporate_bond_holdings_'
base_url_old = 'https://www.ecb.europa.eu/mopo/pdf/CSPPholdings_'
end_url = '.csv'
start = date(2017, 6, 23)
end = date(2021, 4, 10)

cspp = Get_CSPP(start,end, base_url_old, base_url_new, end_url)
cspp_data = cspp.cssp_collect()

In [ ]:
cspp_data.shape

(249298, 6)

In [ ]:
cspp_data.head(10)

,NCB,Date,ISIN,ISSUER,MATURITY DATE,COUPON RATE
0,IT,2017-06-23,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75
1,IT,2017-06-23,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3
2,IT,2017-06-23,XS1144492532,2i Rete Gas S.p.A.,02/01/2020,1.125
3,IT,2017-06-23,XS1571982468,2i Rete Gas S.p.A.,28/08/2026,1.75
4,IT,2017-06-23,XS0859920406,A2A S.p.A.,28/11/2019,4.5
5,IT,2017-06-23,XS0951567030,A2A S.p.A.,10/01/2021,4.375
6,IT,2017-06-23,XS1004874621,A2A S.p.A.,13/01/2022,3.625
7,IT,2017-06-23,XS1195347478,A2A S.p.A.,25/02/2025,1.75
8,IT,2017-06-23,XS1581375182,A2A S.p.A.,16/03/2024,1.25
9,BE,2017-06-23,XS0763122578,ABB Finance B.V.,26/03/2019,2.625


The final output is a dataset of 249.298 individual corporate bonds purchased between 2017 and 2021 by the ECB. 

### Process Historical data of cumulative purchase breakdowns under the CSPP
***
This section includes the import of the historical cumulative purchase file and its preprocessing. Afterwards the information from the historical cumulative purchases is combined with the historical securities held. In particular it is assumed that the change of cumulative holdings in the primary and secondary market each month is equal to the purchase volume of new corporate bonds in that same month. Furthermore due to lack of more specific information it is assumed that each bond is purchased at the same volume. 
***

In [ ]:
# import historic holdings data
data_mv = pd.read_csv('https://www.ecb.europa.eu/mopo/pdf/CSPP_breakdown_history.csv', sep = ',', encoding = 'latin-1', thousands = ',')
data_mv.columns = data_mv.iloc[0]
data_mv = data_mv.drop([0, 59, 60])
data_mv = data_mv.dropna()

# transform total holdings to numeric
setlocale(LC_NUMERIC, '')
monthly_holdings = data_mv['Total holdings'].apply(lambda x: atof(x)).diff()
monthly_holdings.loc[1] = data_mv['Total holdings'].apply(lambda x: atof(x)).loc[1]

# create range of months
months = pd.date_range(start = '06/2016', end = '04/2021', freq = "M")

# create dataframe with months and holdings 
data_h = {'MONTHS':  months, 'HOLDINGS': monthly_holdings}
df_h = pd.DataFrame(data_h, columns = ['MONTHS','HOLDINGS'])

# add columns with just months 
df_h['MONTH_YEAR'] = df_h['MONTHS'].dt.to_period('M') 
cspp_data["Date"] = pd.to_datetime(cspp_data["Date"])
cspp_data['MONTH_YEAR'] = cspp_data['Date'].dt.to_period('M')

# merge holdings and CSPP data
df_comp = cspp_data.merge(df_h, on = "MONTH_YEAR")

# add total monthly bond counts
count_data = df_comp.groupby(["MONTH_YEAR"]).count().reset_index()
count_data_m = count_data[['MONTH_YEAR', 'MATURITY DATE']]
count_data_m = count_data_m.rename(columns = {"MATURITY DATE": "COUNT"})

# add holdings of each bond in euro (assumption: the ecb holds the same amount of each bond)
df_merge = df_comp.merge(count_data_m, on = "MONTH_YEAR")
df_merge['EURO HOLDING'] = (df_merge['HOLDINGS']/df_merge['COUNT'])

In [ ]:
df_merge.head(10)

,NCB,Date,ISIN,ISSUER,MATURITY DATE,COUPON RATE,MONTH_YEAR,MONTHS,HOLDINGS,COUNT,EURO HOLDING
0,IT,2017-06-23,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75,2017-06,2017-06-30,6.79,1934,0.003511
1,IT,2017-06-23,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3,2017-06,2017-06-30,6.79,1934,0.003511
2,IT,2017-06-23,XS1144492532,2i Rete Gas S.p.A.,02/01/2020,1.125,2017-06,2017-06-30,6.79,1934,0.003511
3,IT,2017-06-23,XS1571982468,2i Rete Gas S.p.A.,28/08/2026,1.75,2017-06,2017-06-30,6.79,1934,0.003511
4,IT,2017-06-23,XS0859920406,A2A S.p.A.,28/11/2019,4.5,2017-06,2017-06-30,6.79,1934,0.003511
5,IT,2017-06-23,XS0951567030,A2A S.p.A.,10/01/2021,4.375,2017-06,2017-06-30,6.79,1934,0.003511
6,IT,2017-06-23,XS1004874621,A2A S.p.A.,13/01/2022,3.625,2017-06,2017-06-30,6.79,1934,0.003511
7,IT,2017-06-23,XS1195347478,A2A S.p.A.,25/02/2025,1.75,2017-06,2017-06-30,6.79,1934,0.003511
8,IT,2017-06-23,XS1581375182,A2A S.p.A.,16/03/2024,1.25,2017-06,2017-06-30,6.79,1934,0.003511
9,BE,2017-06-23,XS0763122578,ABB Finance B.V.,26/03/2019,2.625,2017-06,2017-06-30,6.79,1934,0.003511


## Compustat Data
***
Compustat is a database of financial, statistical and market information on active and inactive global companies throughout the world. By means of the Compustat API the companies in the CSPP dataset were matched with the so called "SIC" industry code (Standard Industrial Classification) and the country of origin of the Issuer companies. Any missing values that could not be matched by the database API (approx. 100) were matched manually. The API is not optimized for Python (only R and SPSS) hence the matching has to be made outside of Python. We just match the results file with the CSPP data file. 
***

In [ ]:
# merge with compustat data (countries and SIC codes)
data_compu = pd.read_excel('match_file_matched_compustat.xlsx', skiprows = 1)
data_compu = data_compu.rename(columns = {'Input_Name': 'ISSUER'})
df_merge_compu = df_merge.merge(data_compu, on = "ISSUER")

In [ ]:
df_merge_compu.head(10)

,NCB,Date,ISIN,ISSUER,MATURITY DATE,COUPON RATE,MONTH_YEAR,MONTHS,HOLDINGS,COUNT,EURO HOLDING,Input_Country,gind,gsector,gsubind,sic,spcindcd,spcseccd,Compustat,Country
0,IT,2017-06-23,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75,2017-06,2017-06-30,6.790,1934,0.003511,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy
1,IT,2017-06-23,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3,2017-06,2017-06-30,6.790,1934,0.003511,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy
2,IT,2017-06-23,XS1144492532,2i Rete Gas S.p.A.,02/01/2020,1.125,2017-06,2017-06-30,6.790,1934,0.003511,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy
3,IT,2017-06-23,XS1571982468,2i Rete Gas S.p.A.,28/08/2026,1.75,2017-06,2017-06-30,6.790,1934,0.003511,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy
4,IT,2017-06-30,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75,2017-06,2017-06-30,6.790,1934,0.003511,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy
5,IT,2017-06-30,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3,2017-06,2017-06-30,6.790,1934,0.003511,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy
6,IT,2017-06-30,XS1144492532,2i Rete Gas S.p.A.,02/01/2020,1.125,2017-06,2017-06-30,6.790,1934,0.003511,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy
7,IT,2017-06-30,XS1571982468,2i Rete Gas S.p.A.,28/08/2026,1.75,2017-06,2017-06-30,6.790,1934,0.003511,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy
8,IT,2017-07-07,XS1088274169,2i Rete Gas S.p.A.,16/07/2019,1.75,2017-07,2017-07-31,5.606,3925,0.001428,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy
9,IT,2017-07-07,XS1088274672,2i Rete Gas S.p.A.,16/07/2024,3,2017-07,2017-07-31,5.606,3925,0.001428,IT,101020.0,10.0,10102020.0,1311.0,380.0,935.0,1.0,Italy


## Greenhouse Gas Emissions by Sector 

**Climate Watch**
***
Climate Watch offers yearly greenhouse gas emissions by a sector breakdows and by countries up to the year 2018. The greenhouse gas emissions of the year 2018 in the European Union were used as a reference and matched with the bond issuer companies on the basis of their SIC codes. The matching of SIC codes and sectors as defined by Climate Watch had to be done manually. We use a sector breakdown of greenhouse gas (GHG) emissions in metric tons of carbon dioxide equivalent. The data can be found on the website of Climate Watch, a think-tank, and comprises 12 sectors.
***
**International Energy Agency**
***
We also use a sector breakdown of GHG emissions produced by the International Energy Agency (IEA). The data can be found on the website of the World Resources Institute, another think-tank, and comprises 30 sectors.
***

First the SIC Codes and Sector matches are merged with the full data file.

In [ ]:
# merge with climate watch sectors 
df_merge_compu = df_merge_compu.rename(columns = {'sic': 'SIC'})
data_climatewatch = pd.read_csv('Sector Climate Watch-Climate Watch.csv', sep = ";")
data_climatewatch = data_climatewatch.loc[:, ~data_climatewatch.columns.str.contains('^Unnamed')]
df_merge_cw = df_merge_compu.merge(data_climatewatch, on = "SIC")

# merge with international energy agency 
data_iea = pd.read_csv('Sub-Sector International Energy Agency-International Energy Agency.csv', sep = ";")
data_iea = data_iea.loc[:, ~data_iea.columns.str.contains('^Unnamed')]
df_merge_final = df_merge_cw.merge(data_iea, on = "SIC")

Then the greenhouse gas emissions per sector are added from separate files. 

In [ ]:
# transform climate watch data
data_co2_cw = pd.read_csv('Climate Watch.csv')
data_co2_cw = data_co2_cw[data_co2_cw.Country == 'European Union (27)']
data_co2_cw = data_co2_cw.drop(columns = ['Data source', 'Gas', 'Unit'])
data_co2_cw = data_co2_cw.melt(id_vars=["Country", "Sector"], var_name = "Year", value_name = "GHG_Total_CW")
data_co2_cw = data_co2_cw[data_co2_cw.Year == '2018']
data_co2_cw = data_co2_cw.drop(columns = ['Year', 'Country'])
data_co2_cw = data_co2_cw.rename(columns = {'Sector': 'Climate Watch'})

# merge climate watch data
df = df_merge_final.merge(data_co2_cw, on = "Climate Watch")

In [ ]:
# transform international energy agency data
data_co2_iea = pd.read_excel('International Energy Agency.xlsx')
data_co2_iea = data_co2_iea.rename(columns={'Sub-sector': 'International Energy Agency','Share of global greenhouse gas emissions (%)': 'GHG_IEA_Share' })

# merge international energy agency data
df = df.merge(data_co2_iea, on = "International Energy Agency")

In [ ]:
df.head(10)

,NCB,Date,ISIN,ISSUER,MATURITY DATE,COUPON RATE,MONTH_YEAR,MONTHS,HOLDINGS,COUNT,...,gsubind,SIC,spcindcd,spcseccd,Compustat,Country,Climate Watch,International Energy Agency,GHG_Total_CW,GHG_IEA_Share
0,IT,2017-06-23,XS0859920406,A2A S.p.A.,28/11/2019,4.5,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9
1,IT,2017-06-23,XS0951567030,A2A S.p.A.,10/01/2021,4.375,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9
2,IT,2017-06-23,XS1004874621,A2A S.p.A.,13/01/2022,3.625,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9
3,IT,2017-06-23,XS1195347478,A2A S.p.A.,25/02/2025,1.75,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9
4,IT,2017-06-23,XS1581375182,A2A S.p.A.,16/03/2024,1.25,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9
5,IT,2017-06-30,XS0859920406,A2A S.p.A.,28/11/2019,4.5,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9
6,IT,2017-06-30,XS0951567030,A2A S.p.A.,10/01/2021,4.375,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9
7,IT,2017-06-30,XS1004874621,A2A S.p.A.,13/01/2022,3.625,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9
8,IT,2017-06-30,XS1195347478,A2A S.p.A.,25/02/2025,1.75,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9
9,IT,2017-06-30,XS1581375182,A2A S.p.A.,16/03/2024,1.25,2017-06,2017-06-30,6.79,1934,...,55103010.0,4911.0,705.0,700.0,1.0,Italy,Electricity/Heat,Oil & Natural Gas,1108.45,3.9


In [ ]:
companiesESG = pd.read_excel('companiesESG.xlsx')

In [ ]:
df = pd.merge(df, companiesESG, on='ISSUER')
df.to_csv('Final_Data_Merge_ESG.csv')

In [ ]:
df = pd.read_csv('Final_Data_Merge_ESG.csv') 

In [ ]:
# data types
datadict = pd.DataFrame(df.dtypes)
# missing values
datadict['MissingVal'] = df.isnull().sum()
# unique values
datadict['NUnique']=df.nunique()
# count of variable
datadict['Count']=df.count()
# rename 0 to datatype
datadict = datadict.rename(columns={0:'DataType'})
datadict

,DataType,MissingVal,NUnique,Count
Unnamed: 0,int64,0,178035,178035
Unnamed: 0.1,int64,0,178035,178035
NCB,object,0,6,178035
Date,object,0,195,178035
ISIN,object,0,1283,178035
ISSUER,object,0,259,178035
MATURITY DATE,object,0,2665,178035
COUPON RATE,object,1,160,178034
MONTH_YEAR,object,0,46,178035
MONTHS,object,0,46,178035


**Variable description:**
    
- **NCB:** Two-letter country code for National Central Bank (NCB) at which bond was purchased 
- **Date:** Date at which the European Central Bank (ECB) purchased the bond.
- **ISIN:** International Securities Identification Number, a code that uniquely identifies a specific securities issue.
- **ISSUER:** Issuers is the corporation that is selling bonds or other debt instruments to raise money.
- **Maturiy date:** Bonds can be classified according to their maturity, which is the date when the company has to pay back the principal to investors (governments in this case).
- **Coupon rate:** The coupon rate or yield is the amount that investors (governments) can expect to receive in income as they hold the bond.
- **Month_year:** Month and year at which the ECB bought the bond.
- **Month:** Month at which the ECB bought the bond.
- **Holdings:** Total monthly purchase volume of the ECB's investment portfolio
- **Count:** Total number of bonds purchased by the ECB in specific month
- **Euro holding:** Purchase volume of each bond in specific month (in mio euros). We assume the ECB holds same amount of each bond, as no specific amount bought of each bond is disclosed.
- **Input country:** Country that is purchasing the bond from the ECB.
- **Gind:** Industry code. 
- **Gsector:** Sector code.
- **Gsubind:** Sub-sector code. 
- **SIC:** Industry code. Standard Industrial Classification was created by the U.S. Office of Management and Budget. SIC codes are 4-digit codes which correspond to various business or industrial activities, such as Oil and Gas Extraction or Printing and Publishing Services.
- **spcindcd:** S&P industry sector code.
- **spcseccd:** S&P economic sector code.
- **Compustat:** 
- **Country:** Country of the bond issuer.
- **Climate Watch:** Climate Watch (CW) industry classification.
- **International Energy Agency:** International Energy Agency (IEA) industry classification.
- **GHG_Total_CW:** Total greehouse gas emissions according to Sectors as defined by Climate Watch (CW) in 2018
- **GHG_IEA_Share:** Share of total greenhouse gas emissions produced by sector to which the company belongs according to International Energy Agenvy (IEA)
- **Emissions:** Refinitiv score for emissions (first environmental category, out of 100).
- **Resource use:** Refinitiv score for resource use (second environmental category, out of 100).
- **Innovation:** Refinitiv score for environmental innovation (third environmental category, out of 100). 
- **Total Environment:** Refinitiv score for environment (first of three facets of ESG score, out of 100). Weighted average relative rating of a company based on the reported environment information and the resulting three environment category scores. 